Forecast Data, https://open-meteo.com/en/docs/knmi-api
-7 to +7 days (Past days + Predictions), as historical weather API has a 5 days delay...
Weather features included:


In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timedelta
import time

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define locations
locations = [
    ("Eelde", 53.214469, 6.566481),
    ("DeBilt", 52.108499, 5.180616),
    ("Vlissingen", 51.455677, 3.576488),
    ("Sittard", 50.998299, 5.86291)
]

# Make sure all required weather variables are listed here
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": [lat for _, lat, _ in locations],
    "longitude": [lon for _, _, lon in locations],
    "hourly": [
        "temperature_2m", "cloud_cover", "wind_speed_10m", 
        "diffuse_radiation", "direct_normal_irradiance", "shortwave_radiation", "apparent_temperature"],
    "models": "knmi_seamless",
    "past_days": 7
}

while True:
    fetch_timestamp = datetime.utcnow()
    responses = openmeteo.weather_api(url, params=params)

    # Collect data for all locations
    dataframes = []
    for idx, (location_name, _, _) in enumerate(locations):
        response = responses[idx]
        hourly = response.Hourly()
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
            "diffuse_radiation": hourly.Variables(3).ValuesAsNumpy(),
            "direct_normal_irradiance": hourly.Variables(4).ValuesAsNumpy(),
            "shortwave_radiation": hourly.Variables(5).ValuesAsNumpy(),
            "apparent_temperature": hourly.Variables(6).ValuesAsNumpy(),
            "location": location_name,
            "fetch_timestamp_utc": fetch_timestamp
        }
        dataframes.append(pd.DataFrame(data=hourly_data))

    # Combine all data into a single DataFrame
    final_dataframe = pd.concat(dataframes, ignore_index=True)

    # Save to CSV with timestamped filename in the Data folder
    timestamp_str = fetch_timestamp.isoformat(timespec='minutes')
    csv_filename = f"../Data/pred_weather_data_{timestamp_str}.csv"
    final_dataframe.to_csv(csv_filename, index=False)

    print(f"Data saved to {csv_filename}")

    # Wait for 1 hour before fetching again
    time.sleep(3600)


/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_6236/4083320322.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  fetch_timestamp = datetime.utcnow()


Data saved to ../Data/pred_weather_data_2025-03-12T16:09.csv
